In [10]:
import pysam
import difflib

In [11]:
REF_FASTA = '../insilico-builder/test_data/ref_38/hg38.fa'

In [12]:
fasta = pysam.FastaFile(REF_FASTA)

In [196]:
ref = 'TCCCTATGCAGAGTAAGGGGAGATGGAAGAAAC'
alt = 'TTCTGATGGGGGAGATGGAAGAACA'
pos = 133662439
chrom = '10'
padding = 1

before = fasta.fetch(chrom, pos-padding-1, pos-1).lower()
middle = fasta.fetch(chrom, pos-1, pos+len(ref)-1).upper()
if middle != ref:
    raise Exception('Reference mismatch')
after = fasta.fetch(chrom, pos+len(ref)-1, pos+len(ref)+padding-1).lower()

old = before + middle + after
new = before + alt + after
print(old)
print(new)

diff = difflib.ndiff(old.upper(), new.upper())
print('|'.join(diff))
diff = difflib.ndiff(old.upper(), new.upper())


def complete_diff(chrom, pos, ref, alt, previous_seq):
    if len(ref) > 1 and len(alt) > 1:
        min_len = min(len(ref), len(alt))
        for i in range(min_len):
            if ref[i] != alt[i]:
                print(f'{chrom}\t{pos+i}\t{ref[i]}\t{alt[i]}')
        if len(ref) == len(alt):
            return
        previous_seq = ref[:min_len] + previous_seq
        ref = ref[min_len-1:]
        alt = ref[0] + alt[min_len:]
        pos = pos + i
    if len(alt) > 1:
        for i, previous_char in enumerate(previous_seq[::-1]):
            if previous_char != alt[1]:
                break
        ref = pending_seq[-1-i]
        alt = ref + alt[1:]
        pos = pos - i
    if len(ref) > 1:
        for i, previous_char in enumerate(previous_seq[::-1]):
            if previous_char + ref[:-1] != ref:
                break
        alt = pending_seq[-1-i]
        ref = alt + ref[1:]
        pos = pos - i
    print(f'{chrom}\t{pos}\t{ref}\t{alt}')


variant_pos = pos - padding - 1
ref_seq = ''
alt_seq = ''
pending_seq = ''
for line in diff:
    start_char, _, char = line
    if start_char == ' ':
        if len(ref_seq) > 1 or len(alt_seq) > 1:
            # INS or DEL
            complete_diff(chrom, variant_pos, ref_seq, alt_seq, pending_seq)
            variant_pos += len(ref_seq) - 1
        ref_seq = char
        alt_seq = char
        pending_seq += char
        variant_pos += 1
    elif start_char == '-':
        ref_seq += char
    elif start_char == '+':
        alt_seq += char
    else:
        raise Exception('Unknown diff line' + line)

if len(ref_seq) > 1 or len(alt_seq) > 1:
    complete_diff(chrom, variant_pos, ref_seq, alt_seq, pending_seq)


aTCCCTATGCAGAGTAAGGGGAGATGGAAGAAACg
aTTCTGATGGGGGAGATGGAAGAACAg
  A|  T|+ T|- C|- C|  C|  T|+ G|  A|  T|  G|- C|- A|- G|- A|- G|- T|- A|- A|  G|  G|  G|  G|  A|  G|  A|  T|  G|  G|  A|  A|  G|  A|  A|+ C|  A|- C|  G
10	133662440	C	T
10	133662440	TC	T
10	133662443	T	TG
10	133662446	GCAGAGTAA	G
10	133662469	A	AC
10	133662470	AC	A
